In [1]:
import random
import os
import pandas as pd
import numpy as np
import copy
# from IPython.core.debugger import set_trace
# import pixiedust

In [2]:
with open('full_dict_relations.txt', 'r') as f:
    full_dict_relations = eval(f.read())

In [4]:
def best_person(group, last_people, remove = True):
    mid_dict = {}
    for i in last_people:
        sum_1_to_others = 0
        for j in list_groups[group - 1]:
            try:
                sum_1_to_others += full_dict_relations[i][j]
            except:
                pass
            try:
                sum_1_to_others += full_dict_relations[j][i]
            except:
                pass
        mid_dict[i] = sum_1_to_others
    sorted_dict = {k: v for k, v in sorted(mid_dict.items(), key=lambda item: item[1], reverse = True)}
    a = list(sorted_dict.keys())[0]
    if remove:
        last_people.remove(a)
#         print(a)
    return a

In [5]:
# получает номер группы, возвращает сумму
def sum_relation(group, list_groups):
    all_sum = 0
    for i in list_groups[group - 1]:
        for j in list_groups[group - 1]:
            try:
                all_sum += full_dict_relations[i][j]
            except:
                pass
            try:
                all_sum += full_dict_relations[j][i]
            except:
                pass
    return all_sum

In [6]:
# получает список групп, возвращает общую сумму и численность всех групп в сумме.
def sum_all_groups(list_groups):
    all_sum = 0
    population = 0
    for i in range(1, 7):
        population += len(list_groups[i - 1])
        all_sum += sum_relation(i, list_groups)
    return all_sum, population

In [7]:
# получает ключ человека в словаре отношений и номер группы.
# возвращает человека и его вклад в цифрах.
def contribution_one_person(person, num_group):
    sum_contr = 0
    for i in list_groups[num_group - 1]:
        try:
            sum_contr += full_dict_relations[person][i]
        except:
            pass
        try:
            sum_contr += full_dict_relations[i][person]
        except:
            pass
    return (person, sum_contr)

In [192]:
# reset
list_groups = [[],[],[],[],[],[]]
last_people_base = list(full_dict_relations.keys())
last_people = list(full_dict_relations.keys())

In [646]:
# найти шесть лучших пятнадцаток.

In [193]:
for i in range(6):
    dict_10_groups = {}
    for k in last_people_base[:]:     
        list_groups[i] = []
        last_people = last_people_base[:]
        list_groups[i].append(k)
        last_people.remove(k)
        for j in range(14):
            list_groups[i].append(best_person(i + 1, last_people, remove = True))
        a = sum_relation(i + 1, list_groups = list_groups)
        dict_10_groups[a] = list_groups[i] 
    list_groups[i] = [] #зачем?
    list_groups[i] = dict_10_groups[max(dict_10_groups)]

    for people in list_groups[i]:
        last_people_base.remove(people)

In [ ]:
sum_all_groups(list_groups = list_groups)

In [5]:
# их заполнить с перетасовкой

In [10]:
# дать группу человека, его имя и коэф для тюна. Возвращает список рациональных переходов для этого человека.
def allowed_groups(group, name, coeff_min):
    _, current_contr = contribution_one_person(name, group)
    minim_contrib = current_contr / coeff_min
    list_allowed_groups = []
    for i in range(6):
        _, contr = contribution_one_person(name, i + 1)
        if contr > minim_contrib or current_contr == 0:
            list_allowed_groups.append(i + 1)
    if group not in list_allowed_groups:
        list_allowed_groups.append(group)
    return list_allowed_groups

In [11]:
# дать словарь возможных комбинация. Возвращает количество комбинаций.
def num_comb(dictionary):
    list_comb = []
    n_comb = 1
    for key, value in dictionary.items():
        n_comb = n_comb * len(value)
    return n_comb

In [19]:
# работает неплохо. Но группы могут быть произвольной длины. И кодом пытался это предотвратить, выдавал ошибку.
# дать словарь с возможными комбинациями и список групп. Он выдаст новую комбинацию.
def best_combination(dictionary, groups):
    n_comb = num_comb(dictionary)
    print(n_comb)
    dict_comb = {}
    ground_list_groups = copy.deepcopy(groups)
    for i in range(n_comb):
        ground_list_groups = copy.deepcopy(groups)
        k = 0
        for key, value in dictionary.items():
            if len(value) > 1 and k == 0:
                ground_list_groups[value.pop() - 1].append(key)
                k += 1
            else:
                ground_list_groups[value[0] - 1].append(key)
        sum_gr, _ = sum_all_groups(ground_list_groups)
#         print(sum_gr)
        dict_comb[sum_gr] = copy.deepcopy(ground_list_groups)
        
    print(len(dict_comb))  
    dict1 = copy.deepcopy(dict_comb)
    for key, values in dict_comb.items():
        for i in values:
            if len(i) > 30:
                del dict1[key]
    return dict_comb[max(dict_comb)]   

In [188]:
# next level
def best_combination(dictionary, groups):
    n_comb = num_comb(dictionary)
    print(n_comb)
    dict_comb = {}
    help_dict = {}
    new_comb = n_comb
    for key, value in dictionary.items():
        if len(value) > 1:
            new_comb = int(new_comb / len(value))
            help_dict[key] = new_comb
        
    for i in range(n_comb):
        ground_list_groups = copy.deepcopy(groups)
        for key, value in dictionary.items():
            if len(value) > 1:
                number = (i // help_dict[key]) % len(value)
                value_num = value[number]
                ground_list_groups[value_num - 1].append(key)
            else:
                ground_list_groups[value[0] - 1].append(key)
        sum_gr, _ = sum_all_groups(ground_list_groups)
        dict_comb[str(ground_list_groups)] = sum_gr
#         set_trace()
#     print(len(dict_comb))   
    dict1 = copy.deepcopy(dict_comb)
    for key, values in dict_comb.items():
        for i in eval(key):
            if len(i) > 30:
#                 print('len > 30')
                try:
                    del dict1[key]
                except:
                    pass
    return eval(max(dict1, key=dict1.get))

In [437]:
# dict_change_groups

In [13]:
depth = 3
coeff_min = 2
# n_group_for_free = 3

In [194]:
len(last_people)

84

In [ ]:
# делаю лишние копии, потому что у меня уже сдвиг на связанных списках.

In [195]:
while len(last_people) > 0:
#     pull_students = []
    dict_min = {}
    for group in range(6):
        dict_rel = {}
        for i in list_groups[group]:
            name, contr = contribution_one_person(i, group + 1)
            dict_rel[name] = contr
        list_min = []
        for i in sorted(dict_rel.items(), key=lambda item: item[1])[:depth]:
            list_min.append(i[0])
        dict_min[group + 1] = list_min
        for i in list_min:
            list_groups[group].remove(i)
#         pull_students.append(last_min)
    dict_change_groups = {}
    for key, value in dict_min.items():
        for i in value:
            dict_change_groups[i] = allowed_groups(key, i, coeff_min)
    free = last_people.pop()
#     pull_students.append(free)
    dict_change_groups[free] = list(range(1, 7))
#     for i in pull_students:
#     n_comb = num_comb(dict_change_groups)
    new_list_groups = copy.deepcopy(best_combination(dict_change_groups, copy.deepcopy(list_groups)))
    all_sum, pop = sum_all_groups(list_groups = new_list_groups)
    print(all_sum, pop, all_sum / pop)
    list_groups = copy.deepcopy(new_list_groups)
    

6
6
2650.0 91 29.12087912087912
6
6
2654.0 92 28.847826086956523
6
6
2679.0 93 28.806451612903224
6
6
2687.0 94 28.585106382978722
12
12
2697.0 95 28.389473684210525
24
24
2714.0 96 28.270833333333332
6
6
2733.0 97 28.175257731958762
6
6
2754.0 98 28.102040816326532
6
6
2761.0 99 27.88888888888889
6
6
2786.0 100 27.86
6
6
2818.0 101 27.900990099009903
6
6
2871.0 102 28.147058823529413
6
6
2906.0 103 28.21359223300971
6
6
2938.0 104 28.25
6
6
2972.0 105 28.304761904761904
6
6
2973.0 106 28.047169811320753
12
12
2989.0 107 27.934579439252335
24
24
2992.0 108 27.703703703703702
48
48
2998.0 109 27.504587155963304
288
288
3013.0 110 27.39090909090909
576
576
3055.0 111 27.52252252252252
576
576
3141.0 112 28.044642857142858
576
576
3224.0 113 28.530973451327434
576
576
3295.0 114 28.903508771929825
576
576
3341.0 115 29.05217391304348
576
576
3415.0 116 29.439655172413794
576
576
3470.0 117 29.65811965811966
576
576
3522.0 118 29.847457627118644
576
576
3598.0 119 30.235294117647058
576
57

In [185]:
# %%pixie_debugger
dict_min = {}
for group in range(6):
    dict_rel = {}
    for i in list_groups[group]:
        name, contr = contribution_one_person(i, group + 1)
        dict_rel[name] = contr
    list_min = []
    for i in sorted(dict_rel.items(), key=lambda item: item[1])[:depth]:
        list_min.append(i[0])
    dict_min[group + 1] = list_min
    for i in list_min:
        list_groups[group].remove(i)

dict_change_groups = {}
for key, value in dict_min.items():
    for i in value:
        dict_change_groups[i] = allowed_groups(key, i, coeff_min)

free = last_people.pop()
dict_change_groups[free] = list(range(1, 7))
dict1 = copy.deepcopy(dict_change_groups)
# print(dict_change_groups)
new_list_groups = copy.deepcopy(best_combination(dict_change_groups, copy.deepcopy(list_groups)))
all_sum, pop = sum_all_groups(list_groups = new_list_groups)
print(all_sum, pop, all_sum / pop)
list_groups = copy.deepcopy(new_list_groups)

576
576
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
len > 30
le

In [34]:
dict1

{'Barrie Youmans': [1],
 'Leonor Simons': [1],
 'Thomas Munch': [1],
 'Genevive Calle': [2],
 'Hosea Kelley': [2],
 'Gaston Fitts': [2],
 'Donnell Barnett': [3],
 'Sherly Candela': [2, 3],
 'Trudy Barber': [3],
 'Deeanna Wiers': [4],
 'Alton Edelson': [4],
 'Stacie Gillespie': [4],
 'Abraham Callan': [5],
 'Errol Fox': [5],
 'Cora Gautier': [5],
 'Bao Wilde': [6],
 'Kristi Meunier': [6],
 'Trena Sebree': [6],
 'Herschel Neifert': [1, 2, 3, 4, 5, 6]}

In [51]:
n_comb = 12
dictionary = copy.deepcopy(dict1)
help_dict = {}
dict_comb = {}
new_comb = n_comb
for key, value in dictionary.items():
    if len(value) > 1:
        new_comb = int(new_comb / len(value))
        help_dict[key] = new_comb
        
        
for i in range(n_comb):
    ground_list_groups = [[], [], [], [], [], []]
    for key, value in dictionary.items():
        if len(value) > 1:
            number = (i // help_dict[key]) % len(value)
            value_num = value[number]
            ground_list_groups[value_num - 1].append(key)
        else:
            ground_list_groups[value[0] - 1].append(key)
        
        
#         if len(value) > 1:
#             queue = 
#             k = n_comb % queue
#             ground_list_groups[value[k] - 1].append(key)
#         else:
#             ground_list_groups[value[0] - 1].append(key)
    sum_gr, _ = sum_all_groups(ground_list_groups)
    dict_comb[str(ground_list_groups)] = sum_gr
#     dict_comb[sum_gr] = copy.deepcopy(ground_list_groups)
# print(len(dict_comb))

In [130]:
# for key, value in dict_comb.items():
#     print(key)
dict_comb['''[['Barrie Youmans', 'Leonor Simons', 'Thomas Munch', 'Herschel Neifert'], ['Genevive Calle', 'Hosea Kelley', 'Gaston Fitts', 'Sherly Candela'], ['Donnell Barnett', 'Trudy Barber'], ['Deeanna Wiers', 'Alton Edelson', 'Stacie Gillespie'], ['Abraham Callan', 'Errol Fox', 'Cora Gautier'], ['Bao Wilde', 'Kristi Meunier', 'Trena Sebree']]''']

40.0

In [48]:
dict1

{'Barrie Youmans': [1],
 'Leonor Simons': [1],
 'Thomas Munch': [1],
 'Genevive Calle': [2],
 'Hosea Kelley': [2],
 'Gaston Fitts': [2],
 'Donnell Barnett': [3],
 'Sherly Candela': [2, 3],
 'Trudy Barber': [3],
 'Deeanna Wiers': [4],
 'Alton Edelson': [4],
 'Stacie Gillespie': [4],
 'Abraham Callan': [5],
 'Errol Fox': [5],
 'Cora Gautier': [5],
 'Bao Wilde': [6],
 'Kristi Meunier': [6],
 'Trena Sebree': [6],
 'Herschel Neifert': [1, 2, 3, 4, 5, 6]}

In [39]:
help_dict

{'Sherly Candela': 6, 'Herschel Neifert': 1}

In [27]:
for key, value in dict1.items():
    for i in value:
        print(i)
    print(key)
#         if len(value) > 1 and k == 0:
#             ground_list_groups[value.pop() - 1].append(key)
#             k += 1
#         else:
#             ground_list_groups[value[0] - 1].append(key)
#     sum_gr, _ = sum_all_groups(ground_list_groups)

1
Barrie Youmans
1
Leonor Simons
1
Thomas Munch
2
3
Sherly Candela
2
Genevive Calle
2
Hosea Kelley
3
Donnell Barnett
3
5
Herschel Neifert
3
Krystal Martin
4
Deeanna Wiers
4
Alton Edelson
4
Stacie Gillespie
5
Abraham Callan
5
Errol Fox
5
Cora Gautier
6
Bao Wilde
6
Kristi Meunier
6
Trena Sebree
1
2
3
4
5
6
Ludie Gee


In [90]:
free

'Herschel Neifert'

In [81]:
dict_change_groups['huina'] = list(range(1, 7))

In [29]:
dict_change_groups

{'Barrie Youmans': [1],
 'Leonor Simons': [1],
 'Thomas Munch': [1],
 'Genevive Calle': [2],
 'Hosea Kelley': [2],
 'Gaston Fitts': [2],
 'Donnell Barnett': [3],
 'Sherly Candela': [2],
 'Trudy Barber': [3],
 'Deeanna Wiers': [4],
 'Alton Edelson': [4],
 'Stacie Gillespie': [4],
 'Abraham Callan': [5],
 'Errol Fox': [5],
 'Cora Gautier': [5],
 'Bao Wilde': [6],
 'Kristi Meunier': [6],
 'Trena Sebree': [6],
 'Herschel Neifert': [1]}

In [30]:
len(last_people)

79

In [31]:
dict_min

{1: ['Barrie Youmans', 'Leonor Simons', 'Thomas Munch'],
 2: ['Genevive Calle', 'Hosea Kelley', 'Gaston Fitts'],
 3: ['Donnell Barnett', 'Sherly Candela', 'Trudy Barber'],
 4: ['Deeanna Wiers', 'Alton Edelson', 'Stacie Gillespie'],
 5: ['Abraham Callan', 'Errol Fox', 'Cora Gautier'],
 6: ['Bao Wilde', 'Kristi Meunier', 'Trena Sebree']}

In [64]:
dict_change_groups

{'Barrie Youmans': [1],
 'Leonor Simons': [1],
 'Thomas Munch': [1],
 'Genevive Calle': [2],
 'Hosea Kelley': [2],
 'Gaston Fitts': [2],
 'Aileen Kramer': [3],
 'Eryn Knick': [3],
 'Landon Mason': [3],
 'Deeanna Wiers': [4],
 'Alton Edelson': [4],
 'Stacie Gillespie': [1],
 'Abraham Callan': [5],
 'Errol Fox': [5],
 'Cora Gautier': [5],
 'Bao Wilde': [6],
 'Kristi Meunier': [6],
 'Trena Sebree': [1],
 'Donnell Barnett': [1]}

In [48]:
list_groups[5]

['Ruby Lester',
 'Clay Powell',
 'Rory Holmes',
 'Julissa Riggins',
 'William Blind',
 'Manuela George',
 'Earline Mcpherson',
 'Erna Krueger',
 'Bobby Bowers',
 'Shanae Stoughton',
 'Chas French',
 'Rosie Watkins',
 'Bao Wilde',
 'Kristi Meunier',
 'Trena Sebree']

In [196]:
for i in list_groups:
    print(len(i))

30
29
30
29
26
30


In [332]:
contribution_one_person('Aileen Kramer', 2)

('Aileen Kramer', 0)

In [140]:
dict_min = {}
for group in range(6):
    dict_rel = {}
    for i in list_groups[group]:
        name, contr = contribution_one_person(i, 1)
        dict_rel[name] = contr
    list_min = []
    for i in sorted(dict_rel.items(), key=lambda item: item[1])[:depth]:
        list_min.append(i[0])
    dict_min[group + 1] = list_min
dict_change_groups = {}
for key, value in dict_min.items():
    for i in value:
        dict_change_groups[i] = allowed_groups(key, i, coeff_min)
free = last_people[0]
dict_change_groups[free] = list(range(1, 7))

In [524]:
dict_change_groups

{'Mary Clements': [1],
 'Reynaldo Bruce': [1],
 'Laurence Gapinski': [1],
 'Lincoln Good': [2],
 'Jeannine Valone': [2],
 'Israel Arreguin': [2],
 'Amos Finley': [3],
 'Sina Maggio': [3],
 'Emile Ferguson': [3],
 'Pearl Lloyd': [4],
 'Deshawn Tower': [4],
 'Sandra Spradling': [4],
 'Peggie Brent': [5],
 'Katrina Tocco': [5],
 'Jamie Mayer': [5],
 'Clay Powell': [6],
 'Rosie Watkins': [6],
 'Darell Mcglown': [6],
 'Thomas Munch': [1]}

In [210]:
dict1 = {}
for i in list_groups[0]:
    name, contr = contribution_one_person(i, 1)
    dict1[name] = contr

In [258]:
allowed_groups(1, 'Eryn Knick', coeff_min = 2)

[3, 5]

In [226]:
for i in sorted(dict1.items(), key=lambda item: item[1])[:4]:
    print(i[0])

Tracey Duarte
Cris Brundidge
Klara Pagani
Keenan Stewart


In [218]:
sorted(dict1.items(), key=lambda item: item[1])[:5]

[('Tracey Duarte', 23.5),
 ('Cris Brundidge', 23.5),
 ('Klara Pagani', 23.5),
 ('Keenan Stewart', 28.0),
 ('Dewayne Kaiser', 28.0),
 ('Stacy Wilkinson', 28.0),
 ('Joann Huynh', 30.5),
 ('Belia Avey', 30.5),
 ('Mandi Sponsler', 33.0),
 ('Nanette Shepherd', 37.5)]

In [213]:
dict1

{'Mandi Sponsler': 33.0,
 'Tracey Duarte': 23.5,
 'Cris Brundidge': 23.5,
 'Klara Pagani': 23.5,
 'Keenan Stewart': 28.0,
 'Dewayne Kaiser': 28.0,
 'Joann Huynh': 30.5,
 'Nanette Shepherd': 37.5,
 'Belia Avey': 30.5,
 'Stacy Wilkinson': 28.0}

In [ ]:
# посчитать общую сумму.